In [16]:
import os
from PIL import Image, ImageOps, ImageEnhance, ImageDraw, ImageFont
from tqdm import tqdm

# YOLO 모델을 사용한다고 가정 (이 부분은 원래의 YOLO 모델 코드를 기반으로 하세요)
from ultralytics import YOLO

In [22]:
def NP_Detect(filepath):

    # 학습된 YOLO 모델 로드
    model = YOLO('weights/plate_detect.pt')  # 학습된 모델 경로
    
    # 이미지가 있는 폴더 경로 설정 (입력 이미지와 동일 경로에 처리 결과를 저장하도록 수정)
    input_image_path = os.path.abspath(file_path)  # 입력으로 받은 단일 이미지 파일
    output_folder = os.path.abspath('Boxed_crop/')  # 결과 이미지를 저장할 폴더
    cropped_output_folder = os.path.join(output_folder, 'gray/')  # 크롭된 번호판 이미지를 저장할 폴더
    labels_folder = os.path.join(cropped_output_folder, 'labels/')  # YOLO 형식 라벨을 저장할 폴더
    
    # 결과 이미지를 저장할 폴더 생성
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(cropped_output_folder, exist_ok=True)
    os.makedirs(labels_folder, exist_ok=True)

    # 이미지 파일 처리
    if filename.endswith('.jpg') or filename.endswith('.png'):  # 지원하는 이미지 형식
        # 이미지 경로 설정
        output_image_path = os.path.join(output_folder, os.path.basename(filename))
        label_file_path = os.path.join(labels_folder, os.path.basename(filename).replace('.jpg', '.txt'))
    
        # 이미지 불러오기 (PIL 사용)
        pil_image = Image.open(input_image_path)
        width, height = pil_image.size  # PIL 이미지의 크기
    
        # 모델을 사용해 번호판 객체 감지
        results = model(pil_image)
    
        # 라벨 파일 쓰기 위해 준비
        label_lines = []
    
        # 감지된 바운딩 박스 좌표 및 클래스 정보 출력 및 그리기
        draw = ImageDraw.Draw(pil_image)  # PIL의 이미지에 그리기 객체 생성
    
        for idx, box in enumerate(results[0].boxes):
            # 바운딩 박스 좌표 및 클래스 정보
            bbox = box.xyxy[0].cpu().numpy()  # 바운딩 박스 좌표 (x_min, y_min, x_max, y_max)
            class_id = int(box.cls[0].cpu().numpy())  # 클래스 ID
            confidence = box.conf[0].cpu().numpy()  # 신뢰도
    
            # 클래스 ID가 번호판인지 확인 (여기서는 클래스 ID 0번이 번호판이라고 가정)
            if class_id == 0:  # 번호판 클래스 ID
                # 바운딩 박스 좌표 (정수로 변환)
                x_min, y_min, x_max, y_max = map(int, bbox)
                    
                # 바운딩 박스 그리기 (파란색, 두께 2)
                draw.rectangle([x_min, y_min, x_max, y_max], outline="blue", width=2)
    
                # 신뢰도 및 클래스 ID 표시
                label = f"Number Plate {confidence:.2f}"
                draw.text((x_min, y_min - 10), label, fill="blue")
    
                # 크롭된 이미지 저장
                cropped_image = pil_image.crop((x_min, y_min, x_max, y_max))
    
                # 이미지를 회색조로 변환
                gray_image = ImageOps.grayscale(cropped_image)
    
                # 대비 조정 (1.0은 기본값, 2.0은 대비를 2배로 증가)
                enhancer = ImageEnhance.Contrast(gray_image)
                contrast_image = enhancer.enhance(2.0)
    
                cropped_image_path = os.path.join(cropped_output_folder, f"{os.path.splitext(os.path.basename(filename))[0]}_crop_{idx}.jpg")
                contrast_image.save(cropped_image_path)
    
                # YOLO 형식의 라벨 작성
                x_center = (x_min + x_max) / 2 / width
                y_center = (y_min + y_max) / 2 / height
                bbox_width = (x_max - x_min) / width
                bbox_height = (y_max - y_min) / height
                label_lines.append(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
    
        # 라벨 파일 저장 (YOLO 형식)
        with open(label_file_path, 'w') as label_file:
            label_file.writelines(label_lines)
    
        # 결과 이미지 저장 (전체 이미지)
        pil_image.save(output_image_path)

    print(f"객체 추적 및 박싱 완료. 결과 이미지는 {output_folder}, {cropped_output_folder}, {labels_folder} 폴더에 저장되었습니다.")
    return output_image_path, cropped_output_folder, labels_folder

  0%|                                                                                         | 0/1427 [00:00<?, ?it/s]


0: 384x640 (no detections), 23.5ms
Speed: 5.0ms preprocess, 23.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 number_plate, 21.0ms
Speed: 2.0ms preprocess, 21.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


  0%|                                                                                 | 2/1427 [00:00<01:31, 15.63it/s]


0: 384x640 1 number_plate, 21.0ms
Speed: 2.0ms preprocess, 21.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 number_plate, 21.0ms
Speed: 1.0ms preprocess, 21.0ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)


  0%|▏                                                                                | 4/1427 [00:00<01:25, 16.58it/s]

...


0: 384x640 1 number_plate, 19.0ms
Speed: 2.0ms preprocess, 19.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 number_plate, 18.5ms
Speed: 2.0ms preprocess, 18.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


100%|█████████████████████████████████████████████████████████████████████████████▌| 1420/1427 [01:24<00:00, 16.73it/s]


0: 384x640 1 number_plate, 19.0ms
Speed: 1.0ms preprocess, 19.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 number_plate, 19.0ms
Speed: 2.0ms preprocess, 19.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


100%|██████████████████████████████████████████████████████████████████████████████| 1427/1427 [01:24<00:00, 16.86it/s]

객체 추적 및 박싱 완료. 결과 이미지는 output, cropped 및 labels 폴더에 저장되었습니다.


In [ ]:
# 함수 호출 예시
filename = 'path_to_image.jpg'  # 실제 파일 경로 입력
NP_Detect(filenpath)


In [27]:
import torch
import gc

# After your processing is done
gc.collect()
torch.cuda.empty_cache()